# Claude with Automated Code Execution

This notebook demonstrates automated code execution with AWS Bedrock Claude.

In [ ]:
from dotenv import load_dotenv
import os

# Import your custom Claude client
from your_module import ClaudeAPI  # Update with your actual module

# Import the code execution framework
from code_executor import CodeExecutor, PersistentCodeExecutor
from claude_code_integration import ClaudeCodeAgent, SkillBasedAgent

_ = load_dotenv()

## Setup

In [ ]:
# Initialize Claude client
client = ClaudeAPI(
    region_name='eu-west-1',
    profile_arn=os.getenv("CLAUDE_INFERENCE_PROFILE_ARN")
)

## Example 1: Basic Code Execution

In [ ]:
# Create a simple code executor
executor = CodeExecutor(timeout=60, auto_install_packages=True)

In [ ]:
# Test with simple code
test_code = """
import pandas as pd
import numpy as np

# Create sample data
data = {
    'Month': ['Jan', 'Feb', 'Mar', 'Apr', 'May'],
    'Sales': [100, 120, 115, 140, 160]
}

df = pd.DataFrame(data)
print("Sales Data:")
print(df)
print(f"\nAverage Sales: ${df['Sales'].mean():.2f}")

# Save to CSV
df.to_csv('sales_summary.csv', index=False)
print("\nData saved to sales_summary.csv")
"""

result = executor.execute_python_code(test_code)

print(f"Success: {result['success']}")
print(f"\nOutput:\n{result['output']}")
print(f"\nGenerated files: {result['files']}")
print(f"Execution time: {result['execution_time']:.2f}s")

In [ ]:
# Clean up
executor.cleanup()

## Example 2: Claude Code Agent (Automatic Execution)

In [ ]:
# Create agent with automatic code execution
agent = ClaudeCodeAgent(
    claude_client=client,
    auto_execute=True,
    require_confirmation=True,  # Will ask before executing
    timeout=60
)

In [ ]:
# Ask Claude to generate and execute code
result = agent.ask_and_execute(
    prompt="""
    Create a Python script that:
    1. Generates 100 random data points with a trend
    2. Creates a scatter plot with a fitted line
    3. Saves the plot as 'trend_analysis.png'
    4. Prints R-squared value
    """
)

print(f"\n{'='*60}")
print("RESULTS")
print(f"{'='*60}")
print(f"Success: {result['success']}")
print(f"Files generated: {result['files']}")
print(f"\nClaude's response:\n{result['response'][:500]}...")

In [ ]:
# Show execution details
if result['executions']:
    for i, exec_result in enumerate(result['executions'], 1):
        print(f"\nExecution {i}:")
        print(f"  Success: {exec_result['success']}")
        print(f"  Output: {exec_result['output']}")
        if exec_result['error']:
            print(f"  Error: {exec_result['error']}")

In [ ]:
# Display generated image (if any)
from IPython.display import Image, display
import os

workspace = result['executions'][0]['workspace'] if result['executions'] else agent.executor.workspace_dir

for filename in result['files']:
    if filename.endswith('.png'):
        img_path = os.path.join(workspace, filename)
        if os.path.exists(img_path):
            print(f"\nDisplaying {filename}:")
            display(Image(filename=img_path))

## Example 3: Skill-Based Agent

In [ ]:
# Create skill-based agent
skill_agent = SkillBasedAgent(
    claude_client=client,
    skills_directory="./custom_skills",
    auto_execute=True,
    require_confirmation=False,  # Auto-execute without confirmation
    persistent=True,
    workspace_dir="./skill_workspace"
)

In [ ]:
# List available skills
skills = skill_agent.list_skills()
print("Available skills:")
for skill in skills:
    print(f"  - {skill}")

### Use Time Series Analysis Skill

In [ ]:
# Execute with time series skill
result = skill_agent.execute_with_skill(
    skill_name="analyzing-time-series",
    prompt="Analyze this time series data and create diagnostic plots",
    file_path="./data/retail_sales.csv",
    max_tokens=8000
)

print(f"\n{'='*60}")
print("TIME SERIES ANALYSIS RESULTS")
print(f"{'='*60}")
print(f"Success: {result['success']}")
print(f"Files generated: {result['files']}")

In [ ]:
# Display execution results
for i, exec_result in enumerate(result['executions'], 1):
    print(f"\nCode Block {i}:")
    print(f"  Success: {exec_result['success']}")
    if exec_result['output']:
        print(f"  Output:\n{exec_result['output']}")
    if exec_result['error']:
        print(f"  Error: {exec_result['error']}")

In [ ]:
# Display generated plots
workspace = skill_agent.executor.workspace_dir

for filename in result['files']:
    if filename.endswith(('.png', '.jpg')):
        img_path = os.path.join(workspace, filename)
        if os.path.exists(img_path):
            print(f"\n{filename}:")
            display(Image(filename=img_path))

### Use Practice Question Generator Skill

In [ ]:
# Generate practice questions from lecture notes
result = skill_agent.execute_with_skill(
    skill_name="generating-practice-questions",
    prompt="Generate practice questions in Markdown format from these lecture notes",
    file_path="./lecture_notes/notes04.tex",
    max_tokens=4096
)

print(f"\n{'='*60}")
print("PRACTICE QUESTIONS GENERATED")
print(f"{'='*60}")
print(f"Success: {result['success']}")
print(f"Files: {result['files']}")

In [ ]:
# Display the generated questions
from IPython.display import Markdown

if result['files']:
    for filename in result['files']:
        if filename.endswith('.md'):
            content = skill_agent.get_file(filename)
            if content:
                display(Markdown(content.decode('utf-8')))

## Example 4: Multi-Step Workflow

In [ ]:
# Create workflow agent with persistent workspace
workflow_agent = ClaudeCodeAgent(
    claude_client=client,
    persistent=True,
    workspace_dir="./workflow_demo",
    auto_execute=True,
    require_confirmation=False
)

In [ ]:
# Define a multi-step data analysis workflow
workflow_tasks = [
    {
        'prompt': """
        Step 1: Create synthetic sales data
        Generate a CSV file with 12 months of sales data including:
        - Month names
        - Sales values with upward trend
        - Random seasonal variations
        Save as 'monthly_sales.csv'
        """,
        'max_tokens': 2000
    },
    {
        'prompt': """
        Step 2: Analyze and visualize
        Load 'monthly_sales.csv' and create:
        - Line chart showing sales trend
        - Bar chart for monthly comparison
        - Calculate growth rate
        Save plots and print statistics
        """,
        'max_tokens': 3000
    },
    {
        'prompt': """
        Step 3: Generate report
        Create a summary markdown report that includes:
        - Key statistics from the analysis
        - Reference to the generated plots
        - Insights about the trend
        Save as 'sales_report.md'
        """,
        'max_tokens': 2000
    }
]

In [ ]:
# Execute workflow
workflow_results = workflow_agent.execute_workflow(
    tasks=workflow_tasks,
    share_context=True
)

print(f"\n{'='*60}")
print("WORKFLOW COMPLETED")
print(f"{'='*60}")
print(f"Total tasks: {len(workflow_results)}")

for i, result in enumerate(workflow_results, 1):
    print(f"\nTask {i}:")
    print(f"  Success: {result['success']}")
    print(f"  Files: {result['files']}")
    if result['executions']:
        successful = sum(1 for e in result['executions'] if e['success'])
        print(f"  Executions: {successful}/{len(result['executions'])} successful")

In [ ]:
# Display all generated files
all_files = workflow_agent.list_files()
print(f"\nAll generated files ({len(all_files)}):")
for f in all_files:
    print(f"  - {f}")

In [ ]:
# Display the final report
workspace = workflow_agent.executor.workspace_dir

if 'sales_report.md' in all_files:
    content = workflow_agent.get_file('sales_report.md')
    if content:
        print("\n" + "="*60)
        print("FINAL REPORT")
        print("="*60)
        display(Markdown(content.decode('utf-8')))

In [ ]:
# Display generated visualizations
for filename in all_files:
    if filename.endswith(('.png', '.jpg')):
        img_path = os.path.join(workspace, filename)
        if os.path.exists(img_path):
            print(f"\n{filename}:")
            display(Image(filename=img_path))

## Example 5: Extract Code Without Execution

In [ ]:
# Sometimes you want to review code before executing
review_agent = ClaudeCodeAgent(
    claude_client=client,
    auto_execute=False  # Don't execute automatically
)

In [ ]:
# Get code from Claude
result = review_agent.ask_and_execute(
    prompt="Create a function to calculate Fibonacci numbers up to n",
    execute=False
)

print("Claude's response:")
print(result['response'])
print(f"\nFound {len(result['code_blocks'])} code blocks")

In [ ]:
# Review the code
if result['code_blocks']:
    print("\nCode to review:")
    print("="*60)
    for i, code in enumerate(result['code_blocks'], 1):
        print(f"\nBlock {i}:")
        print(code)
        print("="*60)

In [ ]:
# Now execute manually if desired
if result['code_blocks']:
    exec_results = review_agent.executor.execute_code_blocks(result['response'])
    
    for i, exec_result in enumerate(exec_results, 1):
        print(f"\nBlock {i} execution:")
        print(f"  Success: {exec_result['success']}")
        print(f"  Output: {exec_result['output']}")

## Cleanup

In [ ]:
# Clean up all agents (except persistent ones if you want to keep them)
# agent.cleanup()
# workflow_agent.cleanup()  # Comment this out to keep the workflow results

print("Cleanup complete!")

## Safety and Best Practices

### Security Considerations:

1. **Code Review**: Always review generated code before execution, especially for:
   - File system operations
   - Network requests
   - Database operations

2. **Resource Limits**: The executor has:
   - Timeout protection (default 300s)
   - Memory limits (configurable)
   - Blocked dangerous operations

3. **Sandboxing**: Code runs in isolated workspace directories

4. **Import Control**: Configure allowed/blocked imports for your use case

### Performance Tips:

1. **Persistent Workspaces**: Use `PersistentCodeExecutor` for multi-step workflows
2. **Package Caching**: Packages are only installed once per executor
3. **Token Management**: Be mindful of Claude's context window
4. **Batch Operations**: Group related tasks in workflows

### Error Handling:

1. Check `result['success']` before using outputs
2. Review `result['error']` for debugging
3. Use `require_confirmation=True` for critical operations
4. Implement retries for API calls

### Customization:

```python
# Custom executor configuration
custom_executor = CodeExecutor(
    workspace_dir="./my_workspace",
    timeout=600,  # 10 minutes
    max_memory_mb=2048,  # 2GB
    allowed_imports=['pandas', 'numpy', 'matplotlib'],  # Whitelist
    blocked_imports=['os.system', 'subprocess'],  # Blacklist
    auto_install_packages=True
)
```